<a href="https://colab.research.google.com/github/firekebede/Amharic_ads_recommendation/blob/main/amharic_bert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers
!pip install torch

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 81.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 20.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 84.2 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from transformers import AutoTokenizer, AutoModel

# Load the pre-trained English BERT model
english_model_name = 'bert-base-uncased'
english_tokenizer = AutoTokenizer.from_pretrained(english_model_name)
english_model = AutoModel.from_pretrained(english_model_name)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
# Load the Amharic dataset
with open('/content/drive/MyDrive/amh_data.txt', 'r', encoding='utf-8') as f:
    amharic_data = f.read().splitlines()

In [ ]:
# Tokenize the Amharic dataset using the pre-trained English tokenizer
encoded_data = [english_tokenizer(text, padding='max_length', truncation=True, max_length=50) for text in amharic_data]

In [ ]:
import torch

# Set the device to use
device = torch.device('cuda' if torch.cuda.is_available() else 'GPU')

# Load the pre-trained English BERT model
english_model.to(device)

# Fine-tune the pre-trained English BERT model on the Amharic dataset
from torch.utils.data import DataLoader, Dataset

class AmharicDataset(Dataset):
    def __init__(self, encoded_data):
        self.encoded_data = encoded_data

    def __len__(self):
        return len(self.encoded_data)

    def __getitem__(self, idx):
     item_dict = self.encoded_data[idx]
     return {k: torch.tensor(v) for k, v in item_dict.items()}
  
amharic_dataset = AmharicDataset(encoded_data)

from transformers import AdamW, get_linear_schedule_with_warmup

optimizer = AdamW(english_model.parameters(), lr=5e-5)
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=len(amharic_dataset))

english_model.train()

from tqdm import tqdm

dataloader = DataLoader(amharic_dataset, batch_size=16, shuffle=True)

for epoch in range(3):
    for batch in tqdm(dataloader):
        optimizer.zero_grad()
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        outputs = english_model(input_ids, attention_mask=attention_mask)
        losses = outputs.losses
        losses.backward()
        optimizer.step()
        scheduler.step()

  0%|          | 0/854 [00:00<?, ?it/s]


AttributeError: ignored

In [ ]:
# Save the fine-tuned Amharic BERT model
amharic_model_name = 'bert-base-amharic'
amharic_tokenizer = AutoTokenizer.from_pretrained(english_model_name)
amharic_model = AutoModel.from_pretrained(english_model_name)
amharic_model.save_pretrained(amharic_model_name)
amharic_tokenizer.save_pretrained(amharic_model_name)